In [1]:
import requests
import urlparse
import os
import pickle
from bs4 import BeautifulSoup
import urllib2
import re

In [2]:
url = 'http://www.imdb.com/title/tt1430607/'
#http://www.imdb.com/search/title?genres=animation&title_type=feature&sort=moviemeter,asc

response = requests.get(url)

In [3]:
#print response.text

In [4]:
page = response.text

In [5]:
soup = BeautifulSoup(page)

In [6]:
#print soup.prettify()

In [7]:
#print soup.find("a")

In [8]:
#print soup.a.prettify()

In [9]:
for link in soup.find('link'):
    print link

In [10]:
soup.link

<link href="http://www.imdb.com/title/tt1430607/" rel="canonical"/>

In [11]:
url = 'http://www.imdb.com/title/tt2096673/'

In [28]:
def get_imdb_soup(url):
    response = requests.get(url)
    page = response.text
    return BeautifulSoup(page) #for some reason cannot get attributes (ie.title,a,div) from prettify version
    

In [29]:
def get_imdb_header(imdb_soup):
    imdb_header = imdb_soup.title
    return imdb_header


In [30]:
def get_imdb_title(imdb_header):
    header = imdb_header.text
    imdb_title = header.split('(')[0]
    return imdb_title

    
#get_imdb_title(imdb_header)  

In [31]:
def get_imdb_year(imdb_header):
    header = imdb_header.text
    year = re.findall(r'\d+', header)
    imdb_year = int(year[0])
    return imdb_year

    
#imdb_year(imdb_header)

In [224]:
def get_imdb_budget(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Budget:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                bgt = re.findall(regex,n)
                budget =[x.replace(',','') for x in bgt]
                budint =[int(b) for b in budget]
        return budint[0]
                    
                

#get_imdb_budget(mv_soup)      

175000000

In [256]:
def get_imdb_openingwknd(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Opening Weekend:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                opnwknd = re.findall(regex,n)
                openwknd =[x.replace(',','') for x in opnwknd]
                openingwknd =[int(b) for b in openwknd]
        return openingwknd[0]
                    
                

#get_imdb_openingwknd(mv_soup)      

90440272

In [246]:
def get_imdb_gross(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Gross:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                grs = re.findall(regex,n)
                grss =[x.replace(',','') for x in grs]
                gross =[int(b) for b in grss]
        return gross[0]
                    
                
#get_imdb_gross(mv_soup)      

353495492

In [268]:
def get_imdb_runtime(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Runtime:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                print n
                    
                
print get_imdb_runtime(mv_soup)   

None


In [259]:
meow = mv_soup.find_all('div')
for m in meow:
    print m.text




    if (typeof uet == 'function') {
      uet("ns");
    }











 
IMDb




More





All
Titles
TV Episodes
Names
Companies
Keywords
Characters
Quotes
Bios
Plots










Movies,
                TV
                & Showtimes




MOVIES

In Theaters
Showtimes & Tickets
Latest Trailers
Coming Soon
Release Calendar

CHARTS & TRENDS

 Popular Movies & TV
 
Box Office
Oscar Winners
Top 250 Movies
Most Popular by Genre



TV & VIDEO

Fall TV
TV Picks
On Tonight
Top 250 TV
Watch Now on Amazon
DVD & Blu-Ray

SPECIAL FEATURES

Family Entertainment Guide
Best of 2014
X-Ray for Movies & TV
What to Watch







Celebs,
                Events
                & Photos




CELEBS

Born Today
Celebrity News
Most Popular Celebs

PHOTOS

Latest Stills
Latest Posters
Photos We Love



EVENTS

Awards Central
Sundance
SXSW Film Festival
Tribeca
Cannes
Comic-Con
Emmy Awards
Venice Film Festival
Toronto Film Festival
More Popular Events







News &
Community




LATEST HEADLINES


 ‘The Flash’ M

In [22]:
url = 'http://www.imdb.com/title/tt2096673/'

In [247]:
mv_soup = get_imdb_soup(url)
mv_header = get_imdb_header(mv_soup)
mv_title = get_imdb_title(mv_header)
mv_year = get_imdb_year(mv_header)
mv_budget = get_imdb_budget(mv_soup)
mv_openingwknd = get_imdb_openingwknd(mv_soup)
mv_gross = get_imdb_gross(mv_soup)

In [248]:
print mv_header
print mv_title
print mv_year
print mv_budget
print mv_openingwknd
print mv_gross


<title>Inside Out (2015) - IMDb</title>
Inside Out 
2015
175000000
90440272
353495492


175000000


175000000

In [109]:
#mv_soup.find_all('span')